In [1]:
import os
import streamlit as st
import pickle
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.document_loaders import SeleniumURLLoader
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

from dotenv import load_dotenv
load_dotenv()  

c:\Users\kk196\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=os.getenv("google_api_key"),temperature=0.8)

In [4]:
llm("hi how are you?")

'doing well, thank you!'

In [5]:
loader = SeleniumURLLoader(urls=["https://www.promptingguide.ai/research/rag"])

In [6]:
data = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
            separators=['\n\n', '\n', '.', ','],
            chunk_size=10000,chunk_overlap = 1000
        )

In [8]:
text_splitter

In [9]:
docs = text_splitter.split_documents(data)

In [10]:
docs

[Document(page_content='LLM Research Findings\n\nRAG for LLMs\n\nRetrieval Augmented Generation (RAG) for LLMs\n\nThere are many challenges when working with LLMs such as domain knowledge gaps, factuality issues, and hallucination. Retrieval Augmented Generation (RAG) provides a solution to mitigate some of these issues by augmenting LLMs with external knowledge such as databases. RAG is particularly useful in knowledge-intensive scenarios or domain-specific applications that require knowledge that\'s continually updating. A key advantage of RAG over other approaches is that the LLM doesn\'t need to be retrained for task-specific applications. RAG has been popularized recently with its application in conversational agents.\n\nIn this summary, we highlight the main findings and practical insights from the recent survey titled Retrieval-Augmented Generation for Large Language Models: A Survey (opens in a new tab) (Gao et al., 2023). In particular, we focus on the existing approaches, sta

In [11]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [12]:
vectorstore_openai = FAISS.from_documents(docs, embeddings)

In [13]:
vectorstore_openai

In [21]:
db = vectorstore_openai.similarity_search("what is rag")

In [17]:
prompt_template = """
        Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
        provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
        Context:\n {context}?\n
        Question: \n{question}\n

        Answer:
        """

In [18]:
prompt = PromptTemplate(template =prompt_template, input_variables = ["context", "question"])

In [19]:
chain = load_qa_chain(llm=llm, chain_type="stuff", prompt=prompt)

In [22]:
result = chain({"input_documents":db, "question": "what is rag"}, return_only_outputs=True)

c:\Users\kk196\miniconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [23]:
result

{'output_text': "Retrieval-augmented generation (RAG) is a technique that combines the use of large language models (LLMs) with external knowledge sources to improve the performance of natural language generation (NLG) tasks. In this approach, the LLM is first used to generate a context or prompt, which is then used to retrieve relevant information from a knowledge base. This information is then used to augment the LLM's generation, resulting in more accurate and informative output."}